In [1]:
import sys
sys.path.insert(1,'../models')
sys.path.insert(2,'../features')
import create_model
import build_features
import pandas as pd
import numpy as np
import math
import statistics
import csv
from sklearn.metrics import mean_squared_error

In [2]:
data = pd.read_csv("../../data/processed/rushing.csv")
data.head()

,Unnamed: 0,id,Player,Age,Pos,G,GS,Att,Yds,TD,Lng,Y/A,Y/G,Fmb,TD/G,Att/G
0,0,AbduAm00,Ameer Abdullah,22,RB,16,9,143,597,2,36,4.2,37.3,5,0.1,8.9
1,1,AbduAm00,Ameer Abdullah,23,RB,2,2,18,101,0,24,5.6,50.5,0,0.0,9.0
2,2,AbduAm00,Ameer Abdullah,24,RB,14,11,165,552,4,34,3.3,39.4,2,0.3,11.8
3,3,AbduAm00,Ameer Abdullah,25,RB,10,0,1,1,0,1,1.0,0.1,1,0.0,0.1
4,4,AbduAm00,Ameer Abdullah,26,RB,16,0,23,115,0,15,5.0,7.2,1,0.0,1.4


In [3]:
# Get all player ids
players = data.id.unique()

In [4]:
# For each rusher
offense_type = "rushing"
non_rookies = []
yards_error = []
td_error = []
yards = []
tds = []
for player in players:
    # Get their career data
    career_data = build_features.get_model_data(offense_type,player)
    if isinstance(career_data,str):
        continue
    non_rookies.append(player)
    
    # Split data into training and testing data
    numberOfYears = len(career_data.index)
    training_data = career_data.iloc[0:numberOfYears-1]
    testing_data = career_data.iloc[numberOfYears-1]
    
    # Prepare training data
    X_train = training_data[["GS","Att","Y/A","Y/G","TD/G","Att/G"]]
    y_train = training_data[["Yds","TD"]]
    
    # Prepare testing data
    X_test = testing_data[["GS","Att","Y/A","Y/G","TD/G","Att/G"]]
    X_test = np.asarray(X_test)
    X_test = X_test.reshape(-1,6)
    
    y_test = testing_data[["Yds","TD"]]
    y_test = np.asarray(y_test)
    y_test = y_test.reshape(-1,2)
    
    mdl = create_model.create_model(X_train,y_train)
    
    # Make prediction on model
    y_pred = mdl.predict(X_test)
    yards.append(y_pred[0][0])
    tds.append(y_pred[0][1])
    
    try:
        yards_error.append((abs(math.floor(round(y_pred[0][0]) - y_test[0][0]))/math.floor(y_test[0][0]))*100)
    except ZeroDivisionError:
        yards_error.append(0)
    
    try:
        td_error.append((abs(math.floor(round(y_pred[0][1]) - y_test[0][1]))/math.floor(y_test[0][1]))*100)
    except ZeroDivisionError:
        td_error.append(0)
    

In [5]:
print(statistics.mean(yards_error))

14.552737923980185


In [6]:
print(statistics.mean(td_error))

75.47337278106508


In [9]:
filename = "../../data/predictions/rushing.csv"
f = open(filename,'w')
writer = csv.writer(f)
for i in range(len(non_rookies)):
    writer.writerow([non_rookies[i],yards[i],tds[i]])

In [10]:
filename = "../../data/predictions/error_rushing.csv"
f = open(filename,'w')
writer = csv.writer(f)
for i in range(len(non_rookies)):
    writer.writerow([non_rookies[i],yards_error[i],td_error[i]])